In [6]:
import pandas as pd
import numpy as np
import psycopg2

%matplotlib inline

# Calculate Asset Volatility

In [39]:
frequency = "_1d"
file_suffix = 'u18z18' + frequency
time = "11/29 morning signal: "

def all_assts_from_sql(asst1, asst2, asst3, asst4, asst5, asst6, sql_limit_num):
    conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
    asset1 = "SELECT time, close from {} order by id desc limit {}".format(asst1, sql_limit_num)
    asset2 = "SELECT time, close from {} order by id desc limit {}".format(asst2, sql_limit_num)
    asset3 = "SELECT time, close from {} order by id desc limit {}".format(asst3, sql_limit_num)
    asset4 = "SELECT time, close from {} order by id desc limit {}".format(asst4, sql_limit_num)
    asset5 = "SELECT time, close from {} order by id desc limit {}".format(asst5, sql_limit_num)
    asset6 = "SELECT time, close from {} order by id desc limit {}".format(asst6, sql_limit_num)

    df1 = pd.read_sql(asset1, con=conn)
    df2 = pd.read_sql(asset2, con=conn)
    df3 = pd.read_sql(asset3, con=conn)
    df4 = pd.read_sql(asset4, con=conn)
    df5 = pd.read_sql(asset5, con=conn)
    df6 = pd.read_sql(asset6, con=conn)
    conn.close()
    return df1, df2, df3, df4, df5, df6

In [11]:
ada, bch, eos, eth, ltc, trx = all_assts_from_sql("ada"+file_suffix, 
                                                  "bch"+file_suffix, 
                                                  "eos"+file_suffix,
                                                  "eth"+file_suffix, 
                                                  "ltc"+file_suffix, 
                                                  "trx"+file_suffix,
                                                   1000)

ada.sort_values('time', inplace=True)
eth.sort_values('time', inplace=True)
bch.sort_values('time', inplace=True)
ltc.sort_values('time', inplace=True)
eos.sort_values('time', inplace=True)
trx.sort_values('time', inplace=True)

ada_pct = ada['close'].pct_change()
eth_pct = eth['close'].pct_change()
bch_pct = bch['close'].pct_change()
ltc_pct = ltc['close'].pct_change()
eos_pct = eos['close'].pct_change()
trx_pct = trx['close'].pct_change()

ada_max_vol = ada_pct.max()
eth_max_vol = eth_pct.max()
bch_max_vol = bch_pct.max()
ltc_max_vol = ltc_pct.max()
eos_max_vol = eos_pct.max()
trx_max_vol = trx_pct.max()

print ("ADA 最大杠杆不应超过： ", 1/ada_max_vol, "倍")
print ("ETH 最大杠杆不应超过： ", 1/eth_max_vol, "倍")
print ("BCH 最大杠杆不应超过： ", 1/bch_max_vol, "倍")
print ("LTC 最大杠杆不应超过： ", 1/ltc_max_vol, "倍")
print ("EOS 最大杠杆不应超过： ", 1/eos_max_vol, "倍")
print ("TRX 最大杠杆不应超过： ", 1/trx_max_vol, "倍")

ADA 最大杠杆不应超过：  7.057142857142852 倍
ETH 最大杠杆不应超过：  7.909589041095887 倍
BCH 最大杠杆不应超过：  5.298850574712644 倍
LTC 最大杠杆不应超过：  12.882352941176444 倍
EOS 最大杠杆不应超过：  4.591447368421051 倍
TRX 最大杠杆不应超过：  5.784313725490198 倍


In [51]:
# 手动更新实时价格：
ADA_live_price = 0.00000983
ETH_live_price = 0.02880
BCH_live_price = 0.0443
LTC_live_price = 0.00809
EOS_live_price = 0.0007706
TRX_live_price = 0.00000352

ADA_signal = 1 
ETH_signal = -1
BCH_signal = 1
LTC_signal = -1
EOS_signal = 1
TRX_signal = -1

ADA_leverage = 1  
ETH_leverage = 1  
BCH_leverage = 1  
LTC_leverage = 1  
EOS_leverage = 1  
TRX_leverage = 1  

In [60]:
# 定义资金和杠杆规模：
total_capital = 4         # 单位：XBT
capital_utility = 0.05    # 每个策略不超过的总资金百分比 %
            # 杠杆率
# position_number = 6       # 目标资产数量 

In [61]:
def execution_instruction(ADA_live_price, ADA_signal, ADA_leverage,
                          ETH_live_price, ETH_signal, ETH_leverage,
                          BCH_live_price, BCH_signal, BCH_leverage,
                          LTC_live_price, LTC_signal, LTC_leverage,
                          EOS_live_price, EOS_signal, EOS_leverage,
                          TRX_live_price, TRX_signal, TRX_leverage,
                          total_capital, capital_utility, 
                          leverage = 1):
    each_allocation = total_capital*capital_utility
    ada_size = ADA_signal*each_allocation/ADA_live_price*ADA_leverage
    eth_size = ETH_signal*each_allocation/ETH_live_price*ETH_leverage
    bch_size = BCH_signal*each_allocation/BCH_live_price*BCH_leverage
    ltc_size = LTC_signal*each_allocation/LTC_live_price*LTC_leverage
    eos_size = EOS_signal*each_allocation/EOS_live_price*EOS_leverage
    trx_size = TRX_signal*each_allocation/TRX_live_price*TRX_leverage
    
    return ada_size, eth_size, bch_size, ltc_size, eos_size, trx_size

In [62]:
ada_size, eth_size, bch_size, ltc_size, eos_size, trx_size = execution_instruction(ADA_live_price, ADA_signal, ADA_leverage,
                                                                                  ETH_live_price, ETH_signal, ETH_leverage,
                                                                                  BCH_live_price, BCH_signal, BCH_leverage,
                                                                                  LTC_live_price, LTC_signal, LTC_leverage,
                                                                                  EOS_live_price, EOS_signal, EOS_leverage,
                                                                                  TRX_live_price, TRX_signal, TRX_leverage,
                                                                                  total_capital, capital_utility, 
                                                                                  leverage = 1)

In [63]:
print ("交易指令: "+ time, "ADA:", int(ada_size), ", 单位：个")
print ("交易指令: "+ time, "ETH:", int(eth_size), ", 单位：个")
print ("交易指令: "+ time, "BCH:", int(bch_size), ", 单位：个")
print ("交易指令: "+ time, "LTC:", int(ltc_size), ", 单位：个")
print ("交易指令: "+ time, "EOS:", int(eos_size), ", 单位：个")
print ("交易指令: "+ time, "TRX:", int(trx_size), ", 单位：个")

交易指令: 11/29 morning signal:  ADA: 20345 , 单位：个
交易指令: 11/29 morning signal:  ETH: -6 , 单位：个
交易指令: 11/29 morning signal:  BCH: 4 , 单位：个
交易指令: 11/29 morning signal:  LTC: -24 , 单位：个
交易指令: 11/29 morning signal:  EOS: 259 , 单位：个
交易指令: 11/29 morning signal:  TRX: -56818 , 单位：个


In [57]:
#EOS:
129-73

56

In [64]:
3.9261*0.0056

0.021986159999999998